In [30]:
import os
import requests,json
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from dotenv import load_dotenv
from openai import OpenAI

In [31]:
load_dotenv(override=True)
api_key=os.getenv('OPENAI_API_KEY')

In [32]:
openai=OpenAI()

In [33]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
class Website:
    def __init__(self,url):
        self.url=url
        response=requests.get(url,headers=headers)
        soup=BeautifulSoup(response.content,'html.parser')
        self.title=soup.title
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text=soup.body.get_text(separator='\n',strip=True)
        else:
            self.text=""
        links=[link.get('href') for link in soup.find_all('a')]
        self.links=[link for link in links if link]
    def get_content(self):
        return f"Webpage Title:\n{self.title} \n Webpage Content:\n{self.text}\n\n"
        
        
    

In [34]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n\nYou should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [35]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [38]:
def get_links(url):
    website=Website(url)
    response=openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
        {'role':'system','content':link_system_prompt},
        {'role':'user','content':get_links_user_prompt(website)}
    ],
        response_format={"type":"json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)



In [39]:
get_links('https://www.imaginist.ai/')

{'links': [{'type': 'company page', 'url': 'https://www.imaginist.ai'},
  {'type': 'about page', 'url': 'https://www.imaginist.ai/our-story'},
  {'type': 'careers page', 'url': 'https://www.imaginist.ai/careers'}]}

In [43]:
def get_all_details(url):
    result="Landing Page :\n"
    result+= Website(url).get_content()
    links=get_links(url)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link['url']).get_content()
    return result
    


In [44]:
print(get_all_details('https://www.imaginist.ai/'))

Landing Page :
Webpage Title:
<title>Imaginist | Intelligent agents running business</title> 
 Webpage Content:
top of page
Product
Solutions
Our Story
More
Use tab to navigate through the menu items.
Intelligent Automation for Enterprises
Welcome to Imaginist, the readymade intelligent solutions platform for enterprises.
Our Agent Marketplace enables your business to deploy automated workflows at scale, with easy steps and configurable options for your organization. The specialized domain agents collaborate to automate your work at scale.
Collaboration of agents at work
At Imaginist, we  thrive to build next generation intelligent applications. Our platform is designed to orchestrate collaboration of agents at ease. Agents can assimilate information from your diverse enterprise sources and collaborate to perform automation at scale.
Automate your business Usecases
Our platform is designed to be intuitive and user-friendly, with a simple and approachable user interface that makes autom

In [49]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [50]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [51]:
get_brochure_user_prompt("Imaginist", "https://www.imaginist.ai/")

"You are looking at a company called: Imaginist\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding Page :\nWebpage Title:\n<title>Imaginist | Intelligent agents running business</title> \n Webpage Content:\ntop of page\nProduct\nSolutions\nOur Story\nMore\nUse tab to navigate through the menu items.\nIntelligent Automation for Enterprises\nWelcome to Imaginist, the readymade intelligent solutions platform for enterprises.\nOur Agent Marketplace enables your business to deploy automated workflows at scale, with easy steps and configurable options for your organization. The specialized domain agents collaborate to automate your work at scale.\nCollaboration of agents at work\nAt Imaginist, we\xa0 thrive to build next generation intelligent applications. Our platform is designed to orchestrate collaboration of agents at ease. Agents can assimilate information from your diverse enterprise 

In [56]:
def create_brochure(company_name,url):
    response=openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {'role':'system','content':system_prompt},
            {'role':'user','content':get_brochure_user_prompt(company_name,url)}
        ]
    )
    result=response.choices[0].message.content
    return display(Markdown(result))

In [57]:
create_brochure("Imaginist", "https://www.imaginist.ai/")

# Imaginist Brochure

## Welcome to Imaginist

### Intelligent Automation for Enterprises

Imaginist is at the forefront of intelligent solutions for businesses, offering readymade automation tools that streamline processes and enhance productivity. Our Agent Marketplace empowers organizations to deploy automated workflows effortlessly, using specialized domain agents that collaborate to transform the way work is done.

---

## Our Story

Founded in 2024, Imaginist was built on a vision to revolutionize enterprise automation. With our development center located in Trivandrum, we proudly support businesses in optimizing their operations through innovative technology. 

- **2024:** Company established and seed funding received.
- **Goal:** To provide tailored automation solutions to meet the unique needs of each client.
  
Our seasoned team is dedicated to helping you save time and increase operational efficiency, enabling your business to drive growth.

---

## How Can Imaginist Help Your Business?

- **Intuitive User Experience:** Our user-friendly platform simplifies automation for everyone, allowing you to customize and configure pre-built business agents effortlessly.
- **Scalable Solutions:** Deploy custom agents in just minutes and collaboratively scale up automation across your organization.
- **Secure Platform:** Our AI platform secures access to enterprise tools, ensuring the safety of your operations.

---

## Company Culture

At Imaginist, we encourage a culture of innovation and collaboration. Our team is passionate about technology and committed to revolutionizing how enterprises work. We believe in empowering each other and our clients to achieve their goals through intelligent workflows.

---

## Careers at Imaginist

Join us in creating efficient automated solutions for businesses. We are looking for talented individuals who share our passion for technology to be part of our journey.

### Open Positions:

- **Product Manager:** Drive the development of our intelligent agent marketplace while collaborating with engineering and design teams.
- **Full Stack Developer:** Build and maintain our marketplace, ensuring it meets the needs of our customers.

If you’re looking for a dynamic work environment with a commitment to innovation, check out our current openings and become part of the Imaginist team!

---

## Get in Touch

**Location:** Apex, North Carolina, 27502  
**Explore More:** [Visit our website](#)

---

*Join us at Imaginist and be part of the future of intelligent automation!*